In [1]:
import rdflib


In [ ]:
#Tested Queries

'''  
        SELECT ?ufo_shape
        WHERE {
        :ufo :hasShape ?ufo_shape .
        }
'''

In [9]:
def load_graph(graph_file):
    # Create a graph
    g = rdflib.Graph()

    try:
        # Load the data graph
        g.parse(graph_file, format=rdflib.util.guess_format(graph_file))
        print(f"Loaded graph data from {graph_file}")
    except Exception as e:
        print(f"Error loading graph data: {e}")
        return None

    return g

def query_graph(g, query):
    # Perform a query on the graph
    results = g.query(query)
    return results

if __name__ == "__main__":
    # File path
    graph_file = r"C:\Users\belen\Desktop\Université de Lorraine\Second semester\Web_semantics_FinalProject\Web_Semantics_FinalProject\Ontology_building\Populated_graph.ttl"

    # Load graph
    graph = load_graph(graph_file)

    if graph:
        # Define a sample query (modify according to your data)
        sample_query = """
        SELECT ?occupationWitness
        WHERE {
        :witness :hasOccupation ?occupationWitness .
        }
        """

        # Execute the query
        results = query_graph(graph, sample_query)

        # Print the results
        for row in results:
            print(f"{row.occupationWitness}")


Loaded graph data from C:\Users\belen\Desktop\Université de Lorraine\Second semester\Web_semantics_FinalProject\Web_Semantics_FinalProject\Ontology_building\Populated_graph.ttl
Air Traffic Control
Air Traffic Control employee
Air Traffic Controller
Amateur astronomer
Ex-RAF
Ex-RN Cdr
Ex-pilot
Ex-police
Ex-test engineer for Hawker Siddley
Former Air Traffic Controller
Has pilot's licence
Journalist
MoD Guard Service
Part-Time Pilot
Pilot
Police
Police Officer
Police Sgt
Professional Photographer
RAF
RN Air Traffic Control
Retired RAF
Retired merchant seaman
unknown_witness_occupation


In [ ]:
#This code is meant to generate new triples to the graph. So far, we added just empty ones to test the functionality. 
#There is a query to check if its ok

''' 
subject: ufoSighting (this one already exists)
predicate: wasPublished
object: ufoMagazine

'''

In [8]:
def load_graph(graph_file):
    # Create a graph
    g = rdflib.Graph()

    try:
        # Load the data graph
        g.parse(graph_file, format=rdflib.util.guess_format(graph_file))
        print(f"Loaded graph data from {graph_file}")
    except Exception as e:
        print(f"Error loading graph data: {e}")
        return None

    return g

def add_triple(g, subject, predicate, obj):
    # Define the namespace
    ns = rdflib.Namespace("http://www.semanticweb.org/belen/ontologies/2024/5/ufo/")

    # Create RDF terms for the triple
    subj = ns[subject]
    pred = ns[predicate]
    obj = ns[obj]

    # Add the triple to the graph
    g.add((subj, pred, obj))
    print(f"Added triple: ({subject}, {predicate}, {obj})")

def query_graph(g, query):
    # Perform a query on the graph
    results = g.query(query)
    return results

def save_graph(g, graph_file):
    # Serialize the graph back to the file
    g.serialize(destination=graph_file, format=rdflib.util.guess_format(graph_file))
    print(f"Graph saved to {graph_file}")

if __name__ == "__main__":
    # File path
    graph_file = fr"C:\Users\belen\Desktop\Université de Lorraine\Second semester\Web_semantics_FinalProject\Web_Semantics_FinalProject\Ontology_building\Populated_graph.ttl"

    # Load graph
    graph = load_graph(graph_file)

    if graph:
        # Add the triple
        add_triple(graph, "ufoSighting", "wasPublished", "ufoMagazine")

        # Save the updated graph
        save_graph(graph, graph_file)

        # Define a query to check if the triple was added
        sample_query = """
        SELECT ?s ?p ?o
        WHERE {
            ?s :wasPublished ?o .
        }
        """

        # Execute the query
        results = query_graph(graph, sample_query)

        # Print the results
        for row in results:
            print(f"{row.s} {row.p} {row.o}")


Loaded graph data from C:\Users\belen\Desktop\Université de Lorraine\Second semester\Web_semantics_FinalProject\Web_Semantics_FinalProject\Ontology_building\Populated_graph.ttl
Added triple: (ufoSighting, wasPublished, http://www.semanticweb.org/belen/ontologies/2024/5/ufo/ufoMagazine)
Graph saved to C:\Users\belen\Desktop\Université de Lorraine\Second semester\Web_semantics_FinalProject\Web_Semantics_FinalProject\Ontology_building\Populated_graph.ttl
http://www.semanticweb.org/belen/ontologies/2024/5/ufo/ufoSighting None http://www.semanticweb.org/belen/ontologies/2024/5/ufo/ufoMagazine
